In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression

### Search function for each model 

In [23]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", .1, 100, log=True),
    }

    return Ridge(**params)

def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", 1e-4, 1, log=True),
    }

    return Lasso(**params)

def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        "objective": "reg:squarederror",
        "n_estimators": 1000,
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
    }

    return XGBRegressor(**params)

Classifier = (
  Ridge |
  Lasso |
  XGBRegressor )

def instantiate_learner(trial : Trial) -> Classifier:
    algorithm = trial.suggest_categorical(
    'algorithm', ['ridge', 'lasso', 'xgb'])
    
    if algorithm =='ridge':
        model = instantiate_ridge(trial)
    elif algorithm=='lasso':
        model = instantiate_lasso(trial)
    elif algorithm=='xgb':
        model = instantiate_xgb(trial)
    
    return model

In [ ]:
# from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# from category_encoders import WOEEncoder

# Encoder = (
#   OrdinalEncoder |
#   OneHotEncoder 
# )

# def instantiate_encoder(trial : Trial) -> Encoder:
#     method = trial.suggest_categorical(
#     'encoding_method', ['ordinal', 'onehot']
#   )
#     if method=='ordinal':
#     encoder = instantiate_ordinal_encoder(trial)
#     elif method=='onehot':
#     encoder = instantiate_onehot_encoder(trial)
    
#     return encoder

# from sklearn.preprocessing import (
#   StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
# )



In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# def instantiate_numerical_pipeline(trial : Trial) -> Pipeline:
#     pipeline = Pipeline([
#     ('scaler', instantiate_scaler(trial))
#   ])
#     return pipeline

# def instantiate_categorical_function(trial : Trial) -> Pipeline:
#     pipeline = Pipeline([
#     ('encoder', instantiate_encoder(trial))
#   ])
#     return pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = StandardScaler()
    categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_learner(trial)
    
    model = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    return model

In [16]:
def objective(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [28]:
df = pd.read_csv('df.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [19]:
from optuna import create_study

study = create_study(study_name='optimization', direction='maximize')

[I 2024-05-02 10:33:28,101] A new study created in memory with name: optimization


In [29]:
study.optimize(lambda trial: objective(trial, X, y), n_trials=5)

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.329e+11, tolerance: 1.178e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.350e+11, tolerance: 1.127e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_en

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [10, 11] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [5, 9, 14, 22] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2024-05-02 10:37:32,194] Trial 10 finished with value: 0.9182165217554814 and parameters: {'algorithm': 'ridge', 'alpha': 7.369108363877139}. Best is trial 2 with value: 0.9339663751369794.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown cat

In [31]:
study.best_trials

[FrozenTrial(number=2, state=1, values=[0.9339663751369794], datetime_start=datetime.datetime(2024, 5, 2, 10, 34, 49, 117686), datetime_complete=datetime.datetime(2024, 5, 2, 10, 35, 8, 618023), params={'algorithm': 'xgb', 'learning_rate': 0.0058171006274101685, 'max_depth': 7, 'subsample': 0.7791821785361319, 'colsample_bytree': 0.22285902433136467, 'min_child_weight': 13}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'algorithm': CategoricalDistribution(choices=('ridge', 'lasso', 'xgb')), 'learning_rate': FloatDistribution(high=0.1, log=True, low=0.001, step=None), 'max_depth': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.05, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.05, step=None), 'min_child_weight': IntDistribution(high=20, log=False, low=1, step=1)}, trial_id=2, value=None)]